In [ ]:
import requests
import telebot
import time
import json
from datetime import timedelta, datetime

token = ''
chat_id = ''
bot = telebot.TeleBot(token=token)

# Lista para armazenar os jogos já enviados
enviadosht = []
enviadosft = []

while True:
    try:
        print("Buscando partidas...")
        time.sleep(15)

        url = "/api/fixtures/live"
        headers = {"Content-Type": "application/json"}
        response = requests.request("GET", url, headers=headers)
        dic_response = response.json()
        livegames = dic_response
        #verificando id da partida, 
        for game in livegames:
            try:
                fixture_id = game['id'] 
                hometeam = game['localTeam']['name']
                awayteam = game['visitorTeam']['name']
                oddsevents = game['odds']
                oddempate, oddcasa, oddfora = None, None, None
                minute = game['time']['minute']
                status = game['time']['status']

                link_bet365 = f"https://bet365.com/#/AX/K%5E{hometeam.replace(' ', '%20')}/"
                #betobet = f"https://betobet.com/br/sports/search?search={hometeam.replace}/"
                #betbra = f"https://bet-bra.com/sportsbook/search/{hometeam.replace}/"
                #betfair = f"https://www.betfair.com/exchange/plus/search?query={hometeam.replace}/"
                
                # odds no momento
                for odd in oddsevents:
                    if odd['label'] == 'X':
                        oddempate = odd['value']
                    elif odd['label'] == '1':
                        oddcasa = odd['value']
                    elif odd['label'] == '2':
                        oddfora = odd['value']  
            except:
                pass

            
            #verificando estatísticas
            
            stats_url = f"/api/fixture/stats/{fixture_id}"
            stats_response = requests.get(stats_url, headers=headers)
            stats_data = stats_response.json()
            for stats in stats_data.values():
                try:
                    local_stats = stats_data["localTeam"]["stats"]
                    visitor_stats = stats_data["visitorTeam"]["stats"]

                    #goals
                    casascore = local_stats['goals']
                    forascore = visitor_stats['goals']

                    #cartões
                    casaamarelo = local_stats['yellowcards']
                    foraamarelo = visitor_stats['yellowcards']
                    #casavermelho = local_stats['yellowredcards']
                    #foravermelho = visitor_stats['yellowredcards']

                    #ataques e ataques perigosos
                    casaataques = local_stats['attacks']['attacks']
                    foraataques = visitor_stats['attacks']['attacks']
                    casaataquesperigosos = local_stats['attacks']['dangerous_attacks']
                    foraataquesperigosos = visitor_stats['attacks']['dangerous_attacks']

                    #posse de bola e passes
                    #casaposse = local_stats['possessiontime']
                    #foraposse = visitor_stats['possessiontime']
                    #casapassescorretos = local_stats['passes']['percentage']
                    #casapassestotais = local_stats['passes']['total']
                    #casapassescertos = local_stats['passes']['accurate']
                    #forapassescorretos = visitor_stats['passes']['percentage']
                    #forapassestotais = visitor_stats['passes']['total']
                    #forapassescertos = visitor_stats['passes']['accurate']

                    #corners
                    casacantos = local_stats['corners']
                    foracantos = visitor_stats['corners']

                    #chutes
                    local_shots = local_stats["shots"]
                    visitor_shots = visitor_stats["shots"]
                    #chutes totais
                    casachutestotal = local_shots["total"]
                    forachutestotal = visitor_shots["total"]
                    #chutes no gol
                    casachutesnogol = local_shots["ongoal"]
                    forachutesnogol = visitor_shots["ongoal"]
                    #chutesfora
                    casachutesfora = local_shots["offgoal"]
                    forachutesfora = visitor_shots["offgoal"]
                    #chutes dentro da área
                    casachutesnaarea = local_shots["insidebox"]
                    forachutesnaarea = visitor_shots["insidebox"]
                    #chutes fora da área
                    casachutesdefora = local_shots["outsidebox"]
                    forachutesdefora = visitor_shots["outsidebox"]
                    if casachutesnaarea is None:
                        casachutesnaarea = "-"
                    if forachutesnaarea is None:
                        forachutesnaarea = "-"
                    if casachutesdefora is None:
                        casachutesdefora = "-"
                    if forachutesdefora is None:
                        forachutesdefora = "-"
                        
                except:
                    pass


                try: 
                    info_url = f"/api/fixture/info/{fixture_id}"
                    info_response = requests.request("GET", info_url, headers=headers)
                    info_data = info_response.json()
                    for info in info_data.values():
                        emoji_dict = {"D": "E|", "W": "V|", "L": "D|"}
                        casarace = "".join([emoji_dict[result] for result in info_data["localTeam"]["result_string_list"]])
                        forarace = "".join([emoji_dict[result] for result in info_data["visitorTeam"]["result_string_list"]])


                        estadio = info_data['venue']
                        weather = info_data['weather_report']['temperature_celcius']
                        if 'temperature_celcius' in info_data['weather_report']:
                            weather = info_data['weather_report']['temperature_celcius']
                        else:
                            weather = None
                        liga = info_data['league']['name']
                        pais = info_data['league']['country_name']
                except:
                    pass

                    if (minute >= 12) and (minute <= 20):
                        if casascore + forascore == 0:
                            if casachutestotal + forachutestotal >= 6:
                                if fixture_id in enviadosht:
                                    print('Enviado anteriormente')
                                else:
                                    enviadosht.append(fixture_id)
                                    text = f'''
Oportunidade! 🚨 - LiveSoccer
⚠️ Analisar com atenção!

⚽️ <b>Jogo:</b> {hometeam} x {awayteam}
Últimos jogos: {casarace} - {forarace}
Odds atuais: Casa: {oddcasa} -  Emp.: {oddempate} -  Fora: {oddfora}

🕘 <b>Tempo:</b> {minute}' 
🏟 <b>Estádio:</b> {estadio} - Temp.: {weather}°C

🥅 <b>Placar:</b> {casascore} - {forascore}
⛳️ Escanteios: {casacantos} - {foracantos}
🔥 Ataques perigosos: {casaataquesperigosos} - {foraataquesperigosos}
🎯 Chutes totais(no alvo): {casachutestotal}({casachutesnogol}) - {forachutestotal}({forachutesnogol})
🎯 Chutes dentro da área(fora): {casachutesnaarea}({casachutesdefora}) - {forachutesnaarea}({forachutesdefora})

Links da partida:
<a href='{link_bet365}'>Bet365</a> |      
        '''
                                    
                                    print(minute, '|', hometeam, 'x', awayteam, '|', liga, '|' 'Gol HT - Comportamento casa')
                                    bot.send_message(chat_id=chat_id, text=text, parse_mode='HTML', disable_web_page_preview=True)

                    if (minute >= 59) and (minute <= 67):
                        if casascore + forascore == 0:
                            if casachutestotal + forachutestotal >= 12:
                                if casachutesnogol + forachutesnogol >= 5:
                                    if fixture_id in enviadosft:
                                        print('Enviado anteriormente')
                                    else:
                                        enviadosft.append(fixture_id)
                                        text = f'''
Oportunidade! 🚨 - LiverSoccer
⚠️ Analisar com atenção!

⚽️ <b>Jogo:</b> {hometeam} x {awayteam}
Últimos jogos: {casarace} - {forarace}
Odds atuais: Casa: {oddcasa} -  Emp.: {oddempate} -  Fora: {oddfora}

🕘 <b>Tempo:</b> {minute}' 
🏟 <b>Estádio:</b> {estadio} - Temp.: {weather}°C

🥅 <b>Placar:</b> {casascore} - {forascore}
⛳️ Escanteios: {casacantos} - {foracantos}
🔥 Ataques perigosos: {casaataquesperigosos} - {foraataquesperigosos}
🎯 Chutes totais(no alvo): {casachutestotal}({casachutesnogol}) - {forachutestotal}({forachutesnogol})
🎯 Chutes dentro da área(fora): {casachutesnaarea}({casachutesdefora}) - {forachutesnaarea}({forachutesdefora})

Links da partida:
<a href='{link_bet365}'>Bet365</a> |      
        '''
                                    
                                        print(minute, '|', hometeam, 'x', awayteam, '|', liga, '|' 'Gol HT - Comportamento casa')
                                        bot.send_message(chat_id=chat_id, text=text, parse_mode='HTML', disable_web_page_preview=True)
    except:
        pass
        time.sleep(30)